In [17]:
import pydicom
import torch
import os
import sys
import pandas as pd
import numpy as np
import pydicom as DCM
import png
import matplotlib.pyplot as plt
from __future__ import print_function, division
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from PIL import Image, ImageOps, ImageEnhance
from sklearn.preprocessing import LabelEncoder

In [8]:
import warnings
warnings.filterwarnings("ignore")
plt.ion()

In [9]:
# cwd = os.getcwd()
# cwd

In [10]:
# landmarks_frame = pd.read_csv('faces/face_landmarks.csv')
# n = 65
# # accessing nth row 
# img_name = landmarks_frame.iloc[n, 0]
# landmarks = landmarks_frame.iloc[n, 1:].as_matrix()
# landmarks = landmarks.astype('float').reshape(-1, 2)
# # unknown rows but only two columns x and y

# print('Image name: {}'.format(img_name))
# print('Landmarks shape: {}'.format(landmarks.shape))
# print('First 4 Landmarks: {}'.format(landmarks[:4]))
# plt.imshow(io.imread(os.path.join('faces/',img_name)))

In [16]:
def createTrainFrame():
    
    train_mass_csv = pd.read_csv("../CuratedDDSM/Train/mass_case_description_train_set.csv")
    train_calc_csv = pd.read_csv("../CuratedDDSM/Train/calc_case_description_train_set.csv")
    train_calc_csv = train_calc_csv.rename(columns={'breast density': 'breast_density'})
    train_calc_csv['image file path'] = 'Calc/CBIS-DDSM/'+ train_calc_csv['image file path']
    train_calc_csv['ROI mask file path'] ='CalcROI/CBIS-DDSM/' + train_calc_csv['ROI mask file path']
    train_calc_csv['cropped image file path'] ='CalcROI/CBIS-DDSM/' + train_calc_csv['cropped image file path']
    train_mass_csv['image file path'] = 'Mass/CBIS-DDSM/' + train_mass_csv['image file path']
    train_mass_csv['ROI mask file path'] ='MassROI/CBIS-DDSM/' + train_mass_csv['ROI mask file path']
    train_mass_csv['cropped image file path'] ='MassROI/CBIS-DDSM/'+ train_mass_csv['cropped image file path']
    common_col = list(set(train_calc_csv.columns) & set(train_mass_csv.columns))
    train = pd.concat([train_mass_csv[common_col], train_calc_csv[common_col]], ignore_index=True,sort='False')
    train['image file path'] = 'CuratedDDSM/Train/'+train['image file path'] 
    train['ROI mask file path'] = 'CuratedDDSM/Train/'+train['ROI mask file path']
    train['cropped image file path'] = 'CuratedDDSM/Train/'+train['cropped image file path']
    train['pathology_class'] = LabelEncoder().fit_transform(train_df['pathology'])
    
    return train

def createTestFrame():
    test_mass_csv = pd.read_csv("../CuratedDDSM/Test/mass_case_description_test_set.csv")
    test_calc_csv = pd.read_csv("../CuratedDDSM/Test/calc_case_description_test_set.csv")
    test_calc_csv = test_calc_csv.rename(columns={'breast density': 'breast_density'})
    test_calc_csv['image file path'] = 'Calc/CBIS-DDSM/'+ test_calc_csv['image file path']
    test_calc_csv['ROI mask file path'] ='CalcROI/CBIS-DDSM/' + test_calc_csv['ROI mask file path']
    test_calc_csv['cropped image file path'] ='CalcROI/CBIS-DDSM/' + test_calc_csv['cropped image file path']
    test_mass_csv['image file path'] = 'Mass/CBIS-DDSM/' + test_mass_csv['image file path']
    test_mass_csv['ROI mask file path'] ='MassROI/CBIS-DDSM/' + test_mass_csv['ROI mask file path']
    test_mass_csv['cropped image file path'] ='MassROI/CBIS-DDSM/'+ test_mass_csv['cropped image file path']
    common_col = list(set(test_calc_csv.columns) & set(test_mass_csv.columns))
    test = pd.concat([test_mass_csv[common_col], test_calc_csv[common_col]], ignore_index=True,sort='False')
    test['image file path'] = 'CuratedDDSM/Test/'+test['image file path'] 
    test['ROI mask file path'] = 'CuratedDDSM/Test/'+test['ROI mask file path']
    test['cropped image file path'] = 'CuratedDDSM/Test/'+test['cropped image file path']
    train['pathology_class'] = LabelEncoder().fit_transform(train_df['pathology'])
    
    return test

In [18]:
train_df['class'] = LabelEncoder().fit_transform(train_df['pathology']) 
train_df[['pathology', 'class']]
    

,pathology,class
0,MALIGNANT,2
1,MALIGNANT,2
2,BENIGN,0
3,BENIGN,0
4,BENIGN,0
5,MALIGNANT,2
6,MALIGNANT,2
7,MALIGNANT,2
8,BENIGN,0
9,BENIGN,0


In [13]:
train_df = createTrainFrame()
test_df = createTestFrame()
mammogram_dir = '/home/himanshu/CuratedDDSM/'
train_df.to_csv(mammogram_dir+'train.csv')
test_df.to_csv(mammogram_dir+'test.csv')
pd.get_dummies(train_df, columns=['pathology'])
# pytorch handles numeric labels for inbuilt loss functions
train_df

,ROI mask file path,abnormality id,abnormality type,assessment,breast_density,cropped image file path,image file path,image view,left or right breast,pathology,patient_id,subtlety
0,CuratedDDSM/Train/MassROI/CBIS-DDSM/Mass-Train...,1,mass,4,3,CuratedDDSM/Train/MassROI/CBIS-DDSM/Mass-Train...,CuratedDDSM/Train/Mass/CBIS-DDSM/Mass-Training...,CC,LEFT,MALIGNANT,P_00001,4
1,CuratedDDSM/Train/MassROI/CBIS-DDSM/Mass-Train...,1,mass,4,3,CuratedDDSM/Train/MassROI/CBIS-DDSM/Mass-Train...,CuratedDDSM/Train/Mass/CBIS-DDSM/Mass-Training...,MLO,LEFT,MALIGNANT,P_00001,4
2,CuratedDDSM/Train/MassROI/CBIS-DDSM/Mass-Train...,1,mass,4,3,CuratedDDSM/Train/MassROI/CBIS-DDSM/Mass-Train...,CuratedDDSM/Train/Mass/CBIS-DDSM/Mass-Training...,CC,LEFT,BENIGN,P_00004,3
3,CuratedDDSM/Train/MassROI/CBIS-DDSM/Mass-Train...,1,mass,4,3,CuratedDDSM/Train/MassROI/CBIS-DDSM/Mass-Train...,CuratedDDSM/Train/Mass/CBIS-DDSM/Mass-Training...,MLO,LEFT,BENIGN,P_00004,3
4,CuratedDDSM/Train/MassROI/CBIS-DDSM/Mass-Train...,1,mass,4,3,CuratedDDSM/Train/MassROI/CBIS-DDSM/Mass-Train...,CuratedDDSM/Train/Mass/CBIS-DDSM/Mass-Training...,MLO,RIGHT,BENIGN,P_00004,5
5,CuratedDDSM/Train/MassROI/CBIS-DDSM/Mass-Train...,1,mass,4,3,CuratedDDSM/Train/MassROI/CBIS-DDSM/Mass-Train...,CuratedDDSM/Train/Mass/CBIS-DDSM/Mass-Training...,CC,RIGHT,MALIGNANT,P_00009,4
6,CuratedDDSM/Train/MassROI/CBIS-DDSM/Mass-Train...,1,mass,4,3,CuratedDDSM/Train/MassROI/CBIS-DDSM/Mass-Train...,CuratedDDSM/Train/Mass/CBIS-DDSM/Mass-Training...,MLO,RIGHT,MALIGNANT,P_00009,4
7,CuratedDDSM/Train/MassROI/CBIS-DDSM/Mass-Train...,1,mass,4,3,CuratedDDSM/Train/MassROI/CBIS-DDSM/Mass-Train...,CuratedDDSM/Train/Mass/CBIS-DDSM/Mass-Training...,MLO,LEFT,MALIGNANT,P_00015,4
8,CuratedDDSM/Train/MassROI/CBIS-DDSM/Mass-Train...,1,mass,4,2,CuratedDDSM/Train/MassROI/CBIS-DDSM/Mass-Train...,CuratedDDSM/Train/Mass/CBIS-DDSM/Mass-Training...,CC,RIGHT,BENIGN,P_00018,5
9,CuratedDDSM/Train/MassROI/CBIS-DDSM/Mass-Train...,1,mass,4,2,CuratedDDSM/Train/MassROI/CBIS-DDSM/Mass-Train...,CuratedDDSM/Train/Mass/CBIS-DDSM/Mass-Training...,MLO,RIGHT,BENIGN,P_00018,5


In [7]:

# image_path = os.path.join(homedir,train['image file path'][0])
# image_path

In [8]:
# train['image file path'][0]

In [9]:
# train = pd.concat([train_calc_csv[common_col], train_mass_csv[common_col]], ignore_index=True,sort='False')
# train['image file path'] = 'CuratedDDSM/Train/'+train['image file path'] 


In [10]:
labels = set(train_df['pathology'])
labels

{'BENIGN', 'BENIGN_WITHOUT_CALLBACK', 'MALIGNANT'}

In [11]:
# a =  DCM.read_file('../CuratedDDSM/Train/Calc/CBIS-DDSM/Calc-Training_P_00005_RIGHT_CC/08-07-2016-DDSM-23157/1-full mammogram images-38548/000000.dcm')

In [12]:
# index=0
# image = DCM.read_file('../CuratedDDSM/Train/'+train_mass_csv['image file path'][index])
# label = train_mass_csv['pathology'][index]
# print(image.pixel_array.shape)
# image_2d = image.pixel_array.astype(float)
# # plt.imshow(image_2d,cmap='gray')
# image_2d_scaled = (np.maximum(image_2d,0)/ image_2d.max()) * 255.0
# image_2d_scaled = np.uint8(image_2d_scaled)
# # plt.imshow(image_2d_scaled,cmap='gray')
# transformations = \
#             transforms.Compose([transforms.Grayscale(3),
#                                transforms.Resize(512,interpolation=Image.LANCZOS),
#                                transforms.CenterCrop(512)])
# img = Image.fromarray(image_2d_scaled)
# img = transformations(img)
# print(img.getpixel)
# img_np = np.asarray(img)
# plt.imshow(img_np)
# img_np.shape

In [24]:
class MammographyDataset(Dataset):
    """Creating CBIS-DDSM pytorch dataset."""

    def __init__(self, csv_file, root_dir):
        """
        Args:
            csv_file (string): Path to the csv file containing labels.
            root_dir (string): path to CuratedDDSM directory.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        # Transforms
        self.to_tensor = transforms.ToTensor()
        # Read the csv file
        self.frame = pd.read_csv(csv_file)
        # Image Columns
        self.image_arr = np.asarray(self.frame['image file path'])
        # Labels
        self.label_arr = np.asarray(self.frame['pathology'])
        # Calculate Len
        self.data_len = len(self.frame.index)
        # Location of Curated DDSM
        self.root_dir = root_dir
        # Transformations to convert image into 512*512
        self.transformations = \
            transforms.Compose([transforms.Grayscale(1),
                               transforms.Resize(512,interpolation=Image.LANCZOS),
                               transforms.CenterCrop(512)])

    def __len__(self):
        return self.data_len

    def __getitem__(self, index):
        # Open image
        #img_as_img = Image.open(single_image_name)
        image_path = os.path.join(self.root_dir,self.image_arr[index])
        image_dcm= DCM.read_file(image_path)
        
        image_2d = image_dcm.pixel_array.astype(float)
        image_2d_scaled = (np.maximum(image_2d,0)/ image_2d.max()) * 255.0
        image_2d_scaled = np.uint8(image_2d_scaled)
        #Transform image to 512 * 512 
        img = Image.fromarray(image_2d_scaled)
        img_as_img = self.transformations(img)
        # Transform image to tensor
        img_as_tensor = self.to_tensor(img_as_img)

        # Get label(class) of the image based on the cropped pandas column
        image_label = self.label_arr[index]

        return (img_as_tensor, image_label)

torch.Size([1, 512, 512])

In [14]:
# import numpy as np
# import png
# import pydicom

# ds = pydicom.dcmread(path)

# shape = ds.pixel_array.shape

# # Convert to float to avoid overflow or underflow losses.
# image_2d = ds.pixel_array.astype(float)

# # Rescaling grey scale between 0-255
# image_2d_scaled = (np.maximum(image_2d,0) / image_2d.max()) * 255.0

# # Convert to uint
# image_2d_scaled = np.uint8(image_2d_scaled)

# # Write the PNG file
# with open(destination, 'wb') as png_file:
#     w = png.Writer(shape[1], shape[0], greyscale=True)
#     w.write(png_file, image_2d_scaled)